In [5]:
!pip3 install --upgrade pip3
!pip3 install pandas
!pip3 install fuzzywuzzy
!pip3 install sentence-transformers
!pip3 install torch
!pip3 install virtualenv
!pip3 install ipykernel
!pip3 install notebook

Defaulting to user installation because normal site-packages is not writeable
ERROR: Could not find a version that satisfies the requirement pip3 (from versions: none)
ERROR: No matching distribution found for pip3
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation

In [11]:
!pip3 install pandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


In [41]:
import pandas as pd
from fuzzywuzzy import fuzz
from sentence_transformers import SentenceTransformer, util
from collections import defaultdict
import json
import numpy as np

# Option 1: Disable truncation for all columns
pd.set_option('display.max_colwidth', None)

# Option 2: Set maximum column width to a very large number
pd.set_option('display.max_colwidth', 1000)


In [ ]:
# pip33 install virtualenv
# python3 -m venv st 
# source st/bin/activate

# alias python=python3.10  # as env has multiple python versions 3.10 and 3.13
# alias pip3=pip33.10

# pip3 install torch
# pip3 install sentence_transformers

## Functions

In [35]:
import json
import os
import zipfile
from datetime import datetime
from typing import Any, Tuple, List, Dict, Union

import streamlit as st
# Install sentence-transformers: pip install sentence-transformers
try:
    from sentence_transformers import SentenceTransformer, util
except ImportError:
    st.error("Please install sentence-transformers: pip install sentence-transformers")
    raise

###############################################################################
# 1. DataLoader
###############################################################################
class DataLoader:
    """
    Loads pairs of (annotated JSON, LLM-generated JSON) from:
    - Individual files
    - A zip file containing multiple pairs
    
    Each pair is expected to be either:
    - Two files with known naming convention, or
    - A single JSON file containing both annotated and LLM outputs in a structure you define.
    """

    def __init__(self):
        """
        Initialize any required variables or paths here.
        """
        pass

    def load_json_pairs_from_zip(self, zip_path: str) -> List[Tuple[dict, dict]]:
        """
        Load pairs of (annotated JSON, LLM-generated JSON) from a ZIP file.
        
        :param zip_path: Path to the zip file containing JSON pairs.
        :return: A list of tuples (annotated_json, llm_generated_json).
        """
        json_pairs = []

        with zipfile.ZipFile(zip_path, 'r') as z:
            file_list = z.namelist()
            # Example logic: we assume each pair is named like:
            #   annotated_1.json and generated_1.json, annotated_2.json, generated_2.json, etc.
            # Adjust this logic to match your file naming convention or folder structure.
            annotated_files = sorted([f for f in file_list if f.startswith("annotated_")])
            generated_files = sorted([f for f in file_list if f.startswith("generated_")])

            for ann_file, gen_file in zip(annotated_files, generated_files):
                with z.open(ann_file) as af:
                    annotated_data = json.load(af)
                with z.open(gen_file) as gf:
                    generated_data = json.load(gf)
                json_pairs.append((annotated_data, generated_data))
                
        return json_pairs

    def load_json_pairs_from_files(
        self, annotated_path: str, generated_path: str
    ) -> List[Tuple[dict, dict]]:
        """
        Load a single pair of JSONs from two separate files.
        
        :param annotated_path: Path to the ground-truth JSON.
        :param generated_path: Path to the LLM-generated JSON.
        :return: A list containing one tuple: [(annotated_json, llm_generated_json)].
        """
        with open(annotated_path, 'r') as f:
            annotated_data = json.load(f)
        with open(generated_path, 'r') as f:
            generated_data = json.load(f)
        return [(annotated_data, generated_data)]


###############################################################################
# 2. SemanticTextMatcher
###############################################################################
class SemanticTextMatcher:
    """
    Uses a Sentence Transformer model to compare two text fields semantically.
    """
    def __init__(self, model_name: str = "all-MiniLM-L6-v2", similarity_threshold: float = 0.8):
        """
        :param model_name: Name of the sentence-transformers model.
        :param similarity_threshold: A float indicating how close the texts must be to be considered a match.
        """
        self.model = SentenceTransformer(model_name)
        self.similarity_threshold = similarity_threshold

    def are_texts_similar(self, text1: str, text2: str) -> bool:
        """
        Compare two strings semantically using a Sentence Transformer model.
        
        :param text1: First text string.
        :param text2: Second text string.
        :return: True if similarity is above threshold, otherwise False.
        """
        if not text1 and not text2:
            # Both are empty or None => Consider this a match for "empty" scenario
            return True

        embeddings = self.model.encode([text1, text2])
        cosine_sim = util.cos_sim(embeddings[0], embeddings[1]).item()
        return cosine_sim >= self.similarity_threshold


###############################################################################
# 3. ValueComparator
###############################################################################
class ValueComparator:
    """
    Compares values of various data types, including null handling.
    """

    def __init__(
        self,
        text_matcher: SemanticTextMatcher,
        date_format: str = "%Y-%m-%d",
        numeric_tolerance: float = 1e-3
    ):
        """
        :param text_matcher: An instance of SemanticTextMatcher for comparing text fields.
        :param date_format: Format to parse date strings.
        :param numeric_tolerance: Tolerance for numerical comparisons.
        """
        self.text_matcher = text_matcher
        self.date_format = date_format
        self.numeric_tolerance = numeric_tolerance

    def compare_values(self, ann_value: Any, gen_value: Any) -> bool:
        """
        Compare two values with the appropriate method depending on their type.
        
        This method also handles null values:
        - Both null => True Negative condition (but for matching, we treat it as a "match" = True).
        - Null mismatch => automatically not a match.
        
        :param ann_value: Value from annotated JSON.
        :param gen_value: Value from LLM-generated JSON.
        :return: True if they match (TP or TN for the "match" perspective), False otherwise (FP or FN).
        """
        # 1. Both are null => consider them a match
        if ann_value is None and gen_value is None:
            return True

        # 2. One is null, the other is not => mismatch
        if (ann_value is None and gen_value is not None) or (ann_value is not None and gen_value is None):
            return False

        # 3. If both are not None, determine data type and compare
        if isinstance(ann_value, str) and isinstance(gen_value, str):
            # Attempt date parsing first. If fail, do semantic text compare
            if self._is_date_string(ann_value) and self._is_date_string(gen_value):
                return self.compare_dates(ann_value, gen_value)
            else:
                # Semantic text comparison
                return self.text_matcher.are_texts_similar(ann_value, gen_value)
        elif isinstance(ann_value, (int, float)) and isinstance(gen_value, (int, float)):
            return self.compare_numbers(ann_value, gen_value)
        elif isinstance(ann_value, list) and isinstance(gen_value, list):
            return self.compare_lists(ann_value, gen_value)
        elif isinstance(ann_value, dict) and isinstance(gen_value, dict):
            return self.compare_dicts(ann_value, gen_value)
        else:
            # Fallback to direct equality
            return ann_value == gen_value

    def compare_dates(self, date_str1: str, date_str2: str) -> bool:
        """
        Compare two date strings by parsing them into datetime objects.
        
        :param date_str1: First date string.
        :param date_str2: Second date string.
        :return: True if both dates are equal, else False.
        """
        try:
            dt1 = datetime.strptime(date_str1, self.date_format)
            dt2 = datetime.strptime(date_str2, self.date_format)
            return dt1 == dt2
        except ValueError:
            # If parsing fails, revert to direct string comparison or handle otherwise
            return date_str1 == date_str2

    def compare_numbers(self, num1: float, num2: float) -> bool:
        """
        Compare two numbers with a tolerance threshold.
        
        :param num1: First numeric value.
        :param num2: Second numeric value.
        :return: True if the absolute difference is within tolerance, else False.
        """
        return abs(num1 - num2) <= self.numeric_tolerance

    def compare_lists(self, list1: list, list2: list) -> bool:
        """
        Compare two lists. Handles:
         - List of texts (semantic matching, order-agnostic).
         - List of dictionaries (deep comparison).
         - List of numbers or strings (order-agnostic exact or semantic).
        
        In this example, we do a simple approach: 
        - If the list contains dicts, we do a "multiset" comparison of their JSON string forms.
        - If the list contains strings, we attempt a semantic match, order-agnostic.
        
        You can adapt this logic to your exact needs.
        
        :param list1: First list.
        :param list2: Second list.
        :return: True if both lists match, else False.
        """
        if len(list1) != len(list2):
            return False

        # Detect element type by first item
        if len(list1) > 0 and isinstance(list1[0], dict):
            # For dictionary lists, compare as sets of JSON strings (order-agnostic)
            str_set1 = {json.dumps(d, sort_keys=True) for d in list1}
            str_set2 = {json.dumps(d, sort_keys=True) for d in list2}
            return str_set1 == str_set2

        elif len(list1) > 0 and isinstance(list1[0], str):
            # For text lists, do a pairwise semantic matching (order-agnostic).
            # This can get complicated: a simple approach is to see if each item
            # in list1 can be matched to a unique item in list2.
            used_indices = set()
            for item1 in list1:
                match_found = False
                for i, item2 in enumerate(list2):
                    if i not in used_indices and self.text_matcher.are_texts_similar(item1, item2):
                        used_indices.add(i)
                        match_found = True
                        break
                if not match_found:
                    return False
            return True

        else:
            # Fallback: treat as sets for order-agnostic comparison
            return set(list1) == set(list2)

    def compare_dicts(self, dict1: dict, dict2: dict) -> bool:
        """
        Compare two dictionaries by deeply comparing each key's value.
        
        :param dict1: First dictionary.
        :param dict2: Second dictionary.
        :return: True if all matching keys' values compare as True, else False.
        """
        if set(dict1.keys()) != set(dict2.keys()):
            return False
        
        for key in dict1:
            if not self.compare_values(dict1[key], dict2[key]):
                return False
        return True

    @staticmethod
    def _is_date_string(s: str) -> bool:
        """
        Helper to check if a string might represent a date.
        In real usage, you might apply a more robust check or multiple date formats.
        """
        # Simple check: if it matches YYYY-MM-DD
        return True if s and len(s) == 10 and s[4] == '-' and s[7] == '-' else False


###############################################################################
# 4. JSONComparator
###############################################################################
class JSONComparator:
    """
    Compares two JSON objects (annotated vs. LLM-generated) at:
     - Attribute level
     - Record level
     
    Uses ValueComparator to compare individual values.
    """

    def __init__(self, value_comparator: ValueComparator):
        """
        :param value_comparator: An instance of ValueComparator for value-level comparisons.
        """
        self.value_comparator = value_comparator

    def compare_records(
        self, annotated_json: dict, generated_json: dict
    ) -> Dict[str, bool]:
        """
        Compare each attribute in the JSON objects.
        
        :param annotated_json: Annotated (ground truth) JSON.
        :param generated_json: LLM-generated JSON.
        :return: A dictionary {attribute_name: bool} indicating match or mismatch.
        """
        # Gather all keys from both JSONs
        all_keys = set(annotated_json.keys()) | set(generated_json.keys())
        result = {}

        for key in all_keys:
            ann_value = annotated_json.get(key, None)
            gen_value = generated_json.get(key, None)
            match = self.value_comparator.compare_values(ann_value, gen_value)
            result[key] = match
        
        return result


###############################################################################
# 5. MetricTracker
###############################################################################
class MetricTracker:
    """
    Tracks counts for TP, TN, FP, FN during comparisons.
    """

    def __init__(self):
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0

    def update(self, ann_value: Any, gen_value: Any, matched: bool):
        """
        Update TP, FP, FN, TN counts based on annotated value, generated value, and match result.
        
        :param ann_value: Value from annotated JSON.
        :param gen_value: Value from LLM-generated JSON.
        :param matched: Boolean indicating if ann_value == gen_value (under the chosen logic).
        """
        # Null logic
        if ann_value is None and gen_value is None:
            # True Negative if both are null
            self.tn += 1
            return
        if ann_value is None and gen_value is not None:
            # False Positive if annotated is null, but generated is not null
            self.fp += 1
            return
        if ann_value is not None and gen_value is None:
            # False Negative if annotated is not null, but generated is null
            self.fn += 1
            return
        
        # Both are not null
        if matched:
            # True Positive if they match
            self.tp += 1
        else:
            # If they don't match, we need to decide if it's FP or FN
            # For simplicity in attribute-level, we can treat it as:
            #   - If annotated is non-null and we got mismatch => FN
            #   - Also possibly a FP if the predicted value is "different"
            #
            # However, in standard classification:
            #   - mismatch => FN for the annotated class,
            #   - mismatch => FP for the predicted class.
            #
            # For a single attribute, we can either increment both or use a specific approach.
            # We'll do the simpler approach: treat mismatches as both FN and FP, 
            # because we predicted an incorrect value and missed the correct value.
            self.fn += 1
            self.fp += 1

    def calculate_precision(self) -> float:
        """
        Precision = TP / (TP + FP) 
        """
        if (self.tp + self.fp) == 0:
            return 0.0
        return self.tp / (self.tp + self.fp)

    def calculate_recall(self) -> float:
        """
        Recall = TP / (TP + FN)
        """
        if (self.tp + self.fn) == 0:
            return 0.0
        return self.tp / (self.tp + self.fn)

    def reset(self):
        """
        Reset all counts to zero.
        """
        self.tp = 0
        self.fp = 0
        self.fn = 0
        self.tn = 0


###############################################################################
# 6. MetricsCalculator
###############################################################################
class MetricsCalculator:
    """
    Calculates precision, recall at both attribute-level and record-level.
    """

    def __init__(self):
        # We can store results for each record, then compute averages
        self.attribute_level_tracker = []
        self.record_level_tracker = MetricTracker()

    def compare_and_update_metrics(
        self, 
        annotated_json: dict, 
        generated_json: dict, 
        comparison_results: Dict[str, bool]
    ):
        """
        Updates both attribute-level and record-level metrics.
        
        :param annotated_json: Annotated JSON dict.
        :param generated_json: Generated JSON dict.
        :param comparison_results: A dict {key: bool} from JSONComparator.compare_records().
        """
        # For attribute-level, we count match/mismatch per attribute
        for key, matched in comparison_results.items():
            ann_value = annotated_json.get(key, None)
            gen_value = generated_json.get(key, None)

            mt = MetricTracker()
            mt.update(ann_value, gen_value, matched)
            self.attribute_level_tracker.append(mt)

        # For record-level, we say a record is matched only if ALL attributes matched
        record_matched = all(comparison_results.values())
        # We'll do a single update for the entire record:
        if record_matched:
            # True Positive for the record
            self.record_level_tracker.tp += 1
        else:
            # If record is not fully matched, treat it as:
            # - FN (because we missed the correct record?)
            # - FP (because we predicted incorrectly?)
            # This might vary depending on how you define record-level metrics.
            self.record_level_tracker.fn += 1
            self.record_level_tracker.fp += 1

    def compute_attribute_level_metrics(self) -> Tuple[float, float]:
        """
        Aggregate attribute-level metrics across all attribute comparisons.
        
        :return: (precision, recall)
        """
        total_tp = sum(mt.tp for mt in self.attribute_level_tracker)
        total_fp = sum(mt.fp for mt in self.attribute_level_tracker)
        total_fn = sum(mt.fn for mt in self.attribute_level_tracker)

        precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0.0
        recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0.0

        return precision, recall

    def compute_record_level_metrics(self) -> Tuple[float, float]:
        """
        Compute record-level precision and recall.
        
        :return: (precision, recall)
        """
        precision = self.record_level_tracker.calculate_precision()
        recall = self.record_level_tracker.calculate_recall()
        return precision, recall

    def reset(self):
        """
        Reset all metric trackers.
        """
        self.attribute_level_tracker = []
        self.record_level_tracker.reset()


###############################################################################
# 7. StreamlitApp (Example Usage)
###############################################################################
class StreamlitApp:
    """
    Demonstrates how to integrate the above classes in a Streamlit app.
    """

    def __init__(self):
        # Initialize your text matcher, comparator, etc.
        self.text_matcher = SemanticTextMatcher(similarity_threshold=0.8)
        self.value_comparator = ValueComparator(
            text_matcher=self.text_matcher,
            date_format="%Y-%m-%d",
            numeric_tolerance=1e-3
        )
        self.json_comparator = JSONComparator(self.value_comparator)
        self.data_loader = DataLoader()
        self.metrics_calculator = MetricsCalculator()

    def run(self):
        """
        Run the Streamlit application.
        """
        st.title("Precision & Recall for JSON Comparison")

        # File upload
        st.subheader("Upload JSON Files or a ZIP of JSON pairs")
        uploaded_file = st.file_uploader("Upload a ZIP file or two JSON files", type=["zip", "json"], accept_multiple_files=True)

        if uploaded_file:
            # If multiple files are provided, handle logic to parse them as pairs
            json_pairs = []
            
            if len(uploaded_file) == 1 and uploaded_file[0].name.endswith(".zip"):
                # Single ZIP file
                with open("temp.zip", "wb") as f:
                    f.write(uploaded_file[0].getvalue())
                json_pairs = self.data_loader.load_json_pairs_from_zip("temp.zip")

            elif len(uploaded_file) == 2 and all(f.name.endswith(".json") for f in uploaded_file):
                # Two JSON files
                file_contents = []
                for f in uploaded_file:
                    file_contents.append(json.loads(f.read()))
                # We assume the first is annotated, second is generated
                json_pairs = [(file_contents[0], file_contents[1])]
            else:
                st.error("Please upload either a single zip file or exactly two JSON files.")
                return

            # Now we process each pair and calculate metrics
            self.metrics_calculator.reset()

            # Display results in a table
            table_data = []
            for idx, (ann_json, gen_json) in enumerate(json_pairs):
                comparison_results = self.json_comparator.compare_records(ann_json, gen_json)
                self.metrics_calculator.compare_and_update_metrics(ann_json, gen_json, comparison_results)

                # Per-pair attribute-level metrics
                att_precision, att_recall = self._compute_pair_attribute_metrics(ann_json, gen_json, comparison_results)
                rec_match = all(comparison_results.values())
                
                row = {
                    "Pair Index": idx,
                    "Attribute Precision": round(att_precision, 3),
                    "Attribute Recall": round(att_recall, 3),
                    "Record Match?": rec_match
                }
                table_data.append(row)

            # Show table
            st.write("### Metrics per JSON Pair")
            st.table(table_data)

            # Show overall metrics
            attribute_precision, attribute_recall = self.metrics_calculator.compute_attribute_level_metrics()
            record_precision, record_recall = self.metrics_calculator.compute_record_level_metrics()

            st.write("### Overall Attribute-Level Metrics")
            st.write(f"Precision: {attribute_precision:.3f}, Recall: {attribute_recall:.3f}")

            st.write("### Overall Record-Level Metrics")
            st.write(f"Precision: {record_precision:.3f}, Recall: {record_recall:.3f}")

    def _compute_pair_attribute_metrics(self, ann_json, gen_json, comparison_results) -> Tuple[float, float]:
        """
        Compute attribute-level precision and recall for a single pair by building 
        a temporary MetricTracker for each attribute.
        """
        # Summation for the pair
        temp_tps = 0
        temp_fps = 0
        temp_fns = 0

        for key, matched in comparison_results.items():
            ann_value = ann_json.get(key, None)
            gen_value = gen_json.get(key, None)
            mt = MetricTracker()
            mt.update(ann_value, gen_value, matched)
            temp_tps += mt.tp
            temp_fps += mt.fp
            temp_fns += mt.fn

        precision = temp_tps / (temp_tps + temp_fps) if (temp_tps + temp_fps) > 0 else 0.0
        recall = temp_tps / (temp_tps + temp_fns) if (temp_tps + temp_fns) > 0 else 0.0
        return precision, recall


###############################################################################
# To actually run the app: streamlit run this_file.py
###############################################################################
def main():
    """
    Entry point to run this code as a Streamlit app.
    """
    app = StreamlitApp()
    app.run()

if __name__ == "__main__":
    main()

## Main

In [51]:
# Sample data for demonstration
human_data = {
        "Key 1": "Value 1",
        "Key 2": [
            10,
            30
        ],
        "Key 3": [
            {
                "SubKey1": "Text A"
            },
            {
                "SubKey1": "Text Z"
            }
        ],
        "Key 4": "Another Val",
        "Key 5": [
            "Item1",
            "ItemX",
            "ItemY"
        ],
        "Key 6": [
            {
                "SubKey1": "This is a code to test"
            },
            {
                "SubKey1": "Text Y"
            }
        ]
}

llm_data = {
        "Key 1": "Value 1",
        "Key 2": [
            10,
            20
        ],
        "Key 3": [
            {
                "SubKey1": "This is a test"
            },
            {
                "SubKey1": "Text A"
            }
        ],
        "Key 5": [
            "Item1",
            "Item2"
        ],
        "Key 6": [
            {
                "SubKey1": "This is a testing code"
            },
            {
                "SubKey1": "Text D"
            }
        ],
        "Key 7":'Test'
}

In [52]:
print(json.dumps(human_data, indent=4))

{
    "Key 1": "Value 1",
    "Key 2": [
        10,
        30
    ],
    "Key 3": [
        {
            "SubKey1": "Text A"
        },
        {
            "SubKey1": "Text Z"
        }
    ],
    "Key 4": "Another Val",
    "Key 5": [
        "Item1",
        "ItemX",
        "ItemY"
    ],
    "Key 6": [
        {
            "SubKey1": "This is a code to test"
        },
        {
            "SubKey1": "Text Y"
        }
    ]
}


In [53]:
print(json.dumps(llm_data, indent=4))

{
    "Key 1": "Value 1",
    "Key 2": [
        10,
        20
    ],
    "Key 3": [
        {
            "SubKey1": "This is a test"
        },
        {
            "SubKey1": "Text A"
        }
    ],
    "Key 5": [
        "Item1",
        "Item2"
    ],
    "Key 6": [
        {
            "SubKey1": "This is a testing code"
        },
        {
            "SubKey1": "Text D"
        }
    ],
    "Key 7": "Test"
}


In [57]:
# Main execution
# Semantic similarity function using sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2')

try:
    metrics_df = compare_flat_dictionaries(human_data, llm_data)
    metrics_df
except Exception as e:
    print(f"Unexpected error in metrics calculation: {e}")

Average Precision (populated LLM attributes): 70.00%
Average Recall (populated human attributes): 55.56%
Total LLM populated attributes: 5
Total human annotation populated attributes: 6


In [58]:
metrics_df

,Key,Human Value,LLM Value,Match,Similarity,Precision,Recall
0,Key 1,Value 1,Value 1,✓,100.00%,1.0,1.000000
1,Key 2,"[10, 30]","[10, 20]",✓,100.00%,1.0,1.000000
2,Key 3,"[{'SubKey1': 'Text A'}, {'SubKey1': 'Text Z'}]","[{'SubKey1': 'This is a test'}, {'SubKey1': 'Text A'}]",✗,50.00%,0.5,0.500000
3,Key 4,Another Val,,✗,0%,0.0,0.000000
4,Key 5,"['Item1', 'ItemX', 'ItemY']","['Item1', 'Item2']",✗,41.67%,0.5,0.333333
5,Key 6,"[{'SubKey1': 'This is a code to test'}, {'SubKey1': 'Text Y'}]","[{'SubKey1': 'This is a testing code'}, {'SubKey1': 'Text D'}]",✗,50.00%,0.5,0.500000
